In [33]:
import requests
from pyquery import PyQuery as pq
from tqdm import tqdm

import glob
import csv
import os
from collections import OrderedDict

In [34]:
baseurlpre = "https://www.spc.noaa.gov/climo/online/monthly/"
baseurlpost = "_summary.html"
datadir = "data/"
headers = ["Day", "Total", "Torn", "Hail", "Wind"]

In [10]:
stufftopull = []
for myyear in range(0, 19 + 1):    # Ends in 2019
    for mymonth in range(0, 12 + 1):   # Ends in December
        stufftopull.append(f"{myyear:02d}{mymonth:02d}")

In [12]:
for yearmo in tqdm(stufftopull):
    filename = f"{datadir}Monthly{yearmo}.html"
    remoteurl = f"{baseurlpre}{yearmo}{baseurlpost}"
    r = requests.get(remoteurl)
    if r.status_code == 200:   # if we got a good file
        with open(filename, "wb") as f:
            f.write(r.content)


100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [03:12<00:00,  3.07it/s]


In [42]:
masterlist = []
monthlyreports = list(glob.glob(datadir + "Monthly*.html"))
for monthlyreport in monthlyreports:
    with open(monthlyreport, "r") as f:
        html = f.read()
    table = pq(html)("table")[8]
    for row in pq(table)("tr")[1:-1]:    # Skip header and total row
        line = OrderedDict()
        datetemp = pq(pq(row)("td")[0]).text().strip().split("/")
        line['Date'] = f"{datetemp[2]}-{datetemp[0]}-{datetemp[1]}"
        for i, cell in enumerate(pq(row)("td")[1:]):   
            line[headers[i + 1]] = int(pq(cell).text().strip())
        masterlist.append(line)

In [43]:
with open("tallies.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for row in masterlist:
        writer.writerow(list(row.values()))

[OrderedDict([('Date', '2019-05-01'), ('Total', 83), ('Torn', 9), ('Hail', 44), ('Wind', 30)]), OrderedDict([('Date', '2019-05-02'), ('Total', 69), ('Torn', 17), ('Hail', 15), ('Wind', 37)]), OrderedDict([('Date', '2019-05-03'), ('Total', 85), ('Torn', 6), ('Hail', 32), ('Wind', 47)]), OrderedDict([('Date', '2019-05-04'), ('Total', 97), ('Torn', 5), ('Hail', 37), ('Wind', 55)]), OrderedDict([('Date', '2019-05-05'), ('Total', 184), ('Torn', 23), ('Hail', 101), ('Wind', 60)]), OrderedDict([('Date', '2019-05-06'), ('Total', 117), ('Torn', 4), ('Hail', 70), ('Wind', 43)]), OrderedDict([('Date', '2019-05-07'), ('Total', 108), ('Torn', 24), ('Hail', 54), ('Wind', 30)]), OrderedDict([('Date', '2019-05-08'), ('Total', 75), ('Torn', 15), ('Hail', 11), ('Wind', 49)]), OrderedDict([('Date', '2019-05-09'), ('Total', 122), ('Torn', 5), ('Hail', 40), ('Wind', 77)]), OrderedDict([('Date', '2019-05-10'), ('Total', 10), ('Torn', 0), ('Hail', 2), ('Wind', 8)]), OrderedDict([('Date', '2019-05-11'), ('Tot